In [1]:
import fitz
import pandas as pd
import re

## Hoffmann-Kunze

In [3]:
doc = fitz.open("../dat/books/all_books/textbook_Hoffman_Kunze.pdf")

In [6]:
concepts = []
last_x = parent = is_indented = inverted = False
last_concept = ""

for page_num in range(409,416):
    print("Reached page " + str(page_num))
    
    for word in doc[page_num].get_text("blocks", sort="blocks"):
        if re.search(r"^\w \n$", word[4]): continue
        content = word[4].strip()
        if content == "Index": continue
        if content.find("<image") != -1: continue

        current_x = round(word[0] / 4) * 4
        delta_x = current_x - last_x
        last_x = current_x

        if last_x and delta_x > 5 and delta_x < 15:
            is_indented = True
            inverted = (last_concept.find(":") == -1)
            parent = last_concept.replace(":", "").strip()
        if last_x and (delta_x < -5 or delta_x > 15):
            is_indented = False
            parent = False

        info = content.split(", ")
        start = 1
        for idx, i in enumerate(info):
            if re.search(r"^\D", i) is None:
                start = idx
                break
        
        concept = ", ".join(info[0:start]).replace(" (cont,)","").replace(" (cont.)","")
        last_concept = concept.strip()
        pages = info[start:]

        pages = [i.replace(" ","") for i in pages]
        regex = re.compile(r"^\d+")
        
        pages = [regex.findall(i)[0] if len(regex.findall(i)) else None for i in pages]
        if None in pages: continue
        
        if len(info) == 1: continue
        starts_with_of = concept[0:2] == "of"
        ends_with_of = concept[-2:] == "of"
        composed = (concept + " " + parent if (inverted or ends_with_of) and not starts_with_of else parent+" "+concept) if is_indented else concept 
        if parent and concept.find(parent) != -1: composed = concept
        
        print(composed + " on " + " and ".join(pages))
        concepts.append({'concept': composed, 'pages': ", ".join(pages), 'level': 2 if parent else 1})


Reached page 409
Adjoint classical on 148 and 159
Adjoint of transformation on 295
Admissible subspace on 232
Algebra on 117
Algebra of formal power serie� on 119
self-adjoint Algebra on 345
Algebraically closed field on 138
Alternating n-linear function on 144 and 169
Annihilator of subset on 101
Annihilator of sum and intersection on 106
Annihilator of vector ( T-annihilator) on 201 and 202 and 228
Approximation on 283
Associativity on 1
Associativity of matrix multiplication on 19 and 90
Associativity of vector addition on 28
Augmented matrix on 14
Axiom o f  choice on 400
Basis on 41
change of Basis on 92
dual Basis on 99 and 165
for module Basis on 164
ordered Basis on 50
orthonormal Basis on 281
standard b!4�is of l!'n Basis on 41
Bessel's inequality on 287
Bilinear form on 166 and 320 and 359
diagonalization of Bilinear form on 370
group preserving Bilinear form on 379
matrix of Bilinear form on 362
non-degenerate ( non-singular) Bilinear form on 365
positive definite Bilinear f

In [7]:
index_data = pd.DataFrame(concepts)
index_data

,concept,pages,level
0,Adjoint classical,"148, 159",2
1,Adjoint of transformation,295,2
2,Admissible subspace,232,1
3,Algebra,117,1
4,Algebra of formal power serie�,119,2
...,...,...,...
470,quotient of Vector space,397,2
471,of solutions to linear equations,36,1
472,Wedge (exterior) product,"175, 177",1
473,Zero matrix,12,2


In [8]:
index_data.to_csv("../dat/index/textbook_Hoffman_Kunze.csv")

## Nicholson

In [2]:
doc = fitz.open("../dat/books/all_books/Nicholson-OpenLAWA-2019A.pdf")

In [3]:
concepts = []
line_x = 0
last_x = parent = is_indented = inverted = False
last_concept = buffer = content = ""

for page_num in range(686,697):
    print("Reached page " + str(page_num))
    
    words = [word for word in doc[page_num].get_text("words", sort="blocks")]
    for idx, word in enumerate(words):
        print(word)
        if buffer == "": line_x = word[0]
        current_y = round(word[1]/6)
        next_y = round(words[idx+1][1]/6) if idx+1 < len(words) else 0
        buffer += word[4] + " "
        if current_y != next_y:
            content = buffer
            buffer = ""
        else:
            continue
        
        if content[-2:] == ", ":
            buffer = content
            continue

        if re.search(r"\D$", content.strip()) is not None and idx+1 < len(words):
            if round(word[0]/6) - round(words[idx+1][0]/6) > 4:
                buffer = content.replace("- ", "")
                continue
        #print(content)

        if content == "Index": continue
        #if content.find("<image") != -1: continue

        current_x = round(line_x / 6)
        delta_x = current_x - last_x
        last_x = current_x
        #print(delta_x)
        if last_x and delta_x > 1 and delta_x < 3:
            is_indented = True
            inverted = False
            parent = last_concept.replace(":", "").strip()
        if last_x and (delta_x < -1 or delta_x > 3):
            is_indented = False
            parent = False

        info = content.split(", ")
        start = 1
        for idx, i in enumerate(info):
            if re.search(r"^\D", i) is None:
                start = idx
                break
        
        concept = ", ".join(info[0:start]).replace(" (cont,)","").replace(" (cont.)","")
        last_concept = concept.strip()
        pages = info[start:]

        pages = [i.replace(" ","") for i in pages]
        regex = re.compile(r"^\d+")
        
        pages = [regex.findall(i)[0] if len(regex.findall(i)) else None for i in pages]
        if None in pages: continue
        
        if len(info) == 1: continue
        starts_with_of = concept[0:2] == "of"
        ends_with_of = concept[-2:] == "of"
        composed = (concept + " " + parent if (inverted or ends_with_of) and not starts_with_of else parent+", "+concept) if is_indented else concept 
        if parent and concept.find(parent) != -1: composed = concept
        
        print(composed + " on " + " and ".join(pages))
        concepts.append({'concept': composed, 'pages': ", ".join(pages), 'level': 2 if parent else 1})


Reached page 686
(54.000038146972656, 108.20590209960938, 114.63368225097656, 132.99301147460938, 'Index', 0, 0, 0)
(54.000038146972656, 153.852783203125, 63.090702056884766, 165.210205078125, '(i,', 1, 0, 0)
(68.16004180908203, 153.852783203125, 100.89073181152344, 165.210205078125, 'j)-entry,', 1, 0, 1)
(103.2000732421875, 153.852783203125, 113.22140502929688, 164.8914031982422, '35', 1, 0, 2)
(i, j)-entry on 35
(54.000038146972656, 165.7327880859375, 111.3697280883789, 176.7714080810547, '3-dimensional', 1, 1, 0)
(113.5201187133789, 165.7327880859375, 138.21078491210938, 176.7714080810547, 'space,', 1, 1, 1)
(140.64010620117188, 165.7327880859375, 150.66143798828125, 176.7714080810547, '58', 1, 1, 2)
 on 3 and 58
(54.000038146972656, 177.7327880859375, 107.13074493408203, 188.933837890625, 'A-invariance,', 1, 2, 0)
(109.2000732421875, 177.7327880859375, 124.26142120361328, 188.7714080810547, '176', 1, 2, 1)
A-invariance on 176
(54.000038146972656, 189.7327880859375, 92.1307296752929

In [11]:
index_data = pd.DataFrame(concepts)
index_data

,concept,pages,level
0,"(i, j)-entry",35,1
1,,"3, 58",1
2,A-invariance,176,1
3,B-matrix,513,1
4,T-invariant,523,1
...,...,...,...
1663,zero rows,10,1
1664,zero subspace,"263, 339",1
1665,zero transformation,"60, 117, 376",1
1666,zero vector,"263, 330",1


In [12]:
index_data.to_csv("../dat/index/Nicholson-OpenLAWA-2019A.csv")

## Kuttler

In [14]:
doc = fitz.open("../dat/books/all_books/Kuttler-LinearAlgebra-AFirstCourse-2017A.pdf")

In [32]:
concepts = []
line_x = 0
last_x = parent = False
last_concept = buffer = content = ""

for page_num in range(602,606):
    print("Reached page " + str(page_num))
    
    words = [word for word in doc[page_num].get_text("words", sort="blocks")] # to be able to look ahead
    for idx, word in enumerate(words):
        if buffer == "": line_x = word[0]
        
        if word[4].find("INDEX") != -1: 
            buffer = ""
            continue
        
        buffer += word[4] + " "

        current_y = round(word[1]/6)
        next_y = round(words[idx+1][1]/6) if idx+1 < len(words) else 0
        if current_y != next_y:
            content = buffer
            buffer = ""
        else: continue
        #print(content)

        current_x = round(line_x / 6)
        delta_x = current_x - last_x
        last_x = current_x
        #print(delta_x)

        if delta_x > 1 and delta_x < 5:
            parent = last_concept.strip() 
        if delta_x < -1 and delta_x > -5:
            parent = False

        info = content.split(", ")
        start = 1
        for idx, i in enumerate(info):
            if re.search(r"^\D", i) is None:
                start = idx
                break
        
        concept = ", ".join(info[0:start]).strip()
        last_concept = concept

        pages = [i.replace(" ","") for i in info[start:]]
        regex = re.compile(r"^\d+")
        pages = [regex.findall(i)[0] if len(regex.findall(i)) else None for i in pages]
        
        if len(info) == 1: continue

        composed = parent + ", " + concept if parent else concept
        
        print(composed + ": " + ", ".join(pages))
        concepts.append({'concept': composed, 'pages': ", ".join(pages), 'level': 2 if parent else 1})


Reached page 602
∩: 537
∪: 537
\: 537
row-echelon form: 16
reduced row-echelon form: 16
reduced row-echelon form, algorithm: 19
adjugate: 130
back substitution: 13
base case: 540
basic solution: 29
basis: 203, 204, 234, 487
basis, any two same size: 489
box product: 188
cardioid: 443
Cauchy Schwarz inequality: 168
characteristic equation: 349
chemical reactions, balancing: 33
Cholesky factorization, positive deﬁnite: 420
classical adjoint: 130
Cofactor Expansion: 110
cofactor matrix: 130
column space: 211
complex eigenvalues: 369
complex numbers, absolute value: 334
complex numbers, addition: 329
complex numbers, argument: 336
complex numbers, conjugate: 331
complex numbers, conjugate of a product: 336
complex numbers, modulus: 334, 336
complex numbers, multiplication: 330
complex numbers, polar form: 336
complex numbers, roots: 339
complex numbers, standard form: 329
complex numbers, triangle inequality: 334
component form: 162
component of a force: 265, 267
consistent system: 8
coord

In [33]:
index_data = pd.DataFrame(concepts)
print(index_data)
index_data.to_csv("../dat/index/Kuttler-LinearAlgebra-AFirstCourse-2017A.csv")

                      concept     pages  level
0                           ∩       537      1
1                           ∪       537      1
2                           \       537      1
3            row-echelon form        16      1
4    reduced row-echelon form        16      1
..                        ...       ...    ...
297                      work       265      1
298               zero matrix        54      1
299             zero subspace       201      1
300       zero transformation  271, 500      1
301               zero vector       149      1

[302 rows x 3 columns]


## Math 1410

In [34]:
doc = fitz.open("../dat/books/all_books/Math1410_print.pdf")

In [63]:
concepts = []
line_x = indent_lvl = 0
last_x = False
parents = []
last_concept = buffer = content = ""

for page_num in range(402,406):
    print("Reached page " + str(page_num))
    
    words = [word for word in doc[page_num].get_text("words", sort="blocks")] # to be able to look ahead
    for idx, word in enumerate(words):
        if buffer == "": line_x = word[0]
        
        if word[4].find("INDEX") != -1 or word[4].find("A.��") != -1: 
            #print(word[4])
            buffer = ""
            continue
        
        buffer += word[4] + " "

        current_y = round(word[1]/6)
        next_y = round(words[idx+1][1]/6) if idx+1 < len(words) else 0
        if current_y != next_y:
            content = buffer
            buffer = ""
        else: continue
        #print(content)

        current_x = round(line_x / 6)
        delta_x = current_x - last_x
        last_x = current_x
        #print(delta_x)

        if delta_x > 1 and delta_x < 5 or last_concept == "distance":
            indent_lvl += 1
            #print("Indent level: " + str(indent_lvl))
            parents.append(last_concept)
            
        if delta_x < -1 and delta_x > -5 or last_concept == "standard matrix of":
            indent_lvl -= 1
            #print("Indent level: " + str(indent_lvl))
            parents = parents[0:indent_lvl]

        if delta_x < -4 and delta_x > -8:
            indent_lvl -= 2
            #print("Indent level: " + str(indent_lvl))
            parents = parents[0:indent_lvl]

        info = content.split(", ")
        start = False
        for idx, i in enumerate(info):
            if re.search(r"^\D", i) is None:
                start = idx
                break
        if not start:
            concept = ", ".join(info).strip()
            last_concept = concept
            continue
        
        concept = ", ".join(info[0:start]).strip()
        last_concept = concept

        pages = [i.replace(" ","") for i in info[start:]]
        regex = re.compile(r"^\d+")
        pages = [regex.findall(i)[0] if len(regex.findall(i)) else None for i in pages]
        
        if len(info) == 1: continue

        composed = ""
        if indent_lvl == 2: 
            composed += parents[0] + ", " + parents[1] + ", "
        if indent_lvl == 1:
            composed += parents[0] + ", "
        composed += concept
        
        print(composed + ": " + ", ".join(pages))
        concepts.append({'concept': composed, 'pages': ", ".join(pages), 'level': indent_lvl+1})


Reached page 402
∈: 2
/∈: 2
nth Roots of Unity: 49
nth root, of a complex number: 44, 45
x-axis: 12
x-coordinate: 12
y-axis: 12
y-coordinate: 12
abscissa: 12
absolute value: 14
adjugate: 337
antisymmetric: 305
argument, of a complex number, definition of: 35
argument, of a complex number, properties of: 38
augmented matrix: 129
back substitution: 138
basic solution: 177
basic variable: 150
basis: 282
basis, of a null space: 291
Cartesian coordinate plane: 12
Cartesian coordinates: 12
characteristic polynomial: 346, 369
cis(θ): 39
cofactor: 314
cofactor, expansion: 316, 322
cofactor, matrix: 337
column, pivot: 288
column space: 287
column vector: 191
Complex Factorization Theorem: 26
complex number, nth root: 44, 45
complex number, nth Roots of Unity: 49
complex number, argument, definition of: 35
complex number, argument, properties of: 38
complex number, complex conjugate, definition of: 21
complex number, conjugate, properties of: 22
complex number, definition of: 19, 35
complex numb

In [64]:
index_data = pd.DataFrame(concepts)
print(index_data)
index_data.to_csv("../dat/index/Math1410_print.csv")

                           concept   pages  level
0                                ∈       2      1
1                               /∈       2      1
2               nth Roots of Unity      49      1
3    nth root, of a complex number  44, 45      2
4                           x-axis      12      1
..                             ...     ...    ...
297           vectors, unit vector  60, 62      2
298           vectors, zero vector      58      2
299                   Venn Diagram       3      1
300                           work      75      1
301                    zero matrix     196      1

[302 rows x 3 columns]


Let's check some statistics

In [9]:
import os

path = "../dat/index/"
books = []
indices = {}
for book in os.listdir(path):
    index = pd.read_csv(path + book)
    indices[book] = index
    books.append({
        "book": book,
        "index_len": index.shape[0],
        "index_len_lvl1": len(index.loc[index.level==1]),
        "index_len_lvl2": len(index.loc[index.level==2]),
        "index_len_lvl3": len(index.loc[index.level==3])
    })

index_stats = pd.DataFrame(books)
index_stats

,book,index_len,index_len_lvl1,index_len_lvl2,index_len_lvl3
0,Kuttler-LinearAlgebra-AFirstCourse-2017A.csv,302,155,147,0
1,Beezer_First_Course.csv,114,114,0,0
2,textbook_Hoffman_Kunze.csv,475,160,315,0
3,Nicholson-OpenLAWA-2019A.csv,1668,1097,571,0
4,Linear algebra done right — Axler.csv,237,180,57,0
5,Math1410_print.csv,302,110,178,14
6,Hefferon_LinAlgebra.csv,638,325,308,5
7,"linear-Cherey, Denton.csv",173,164,9,0
8,interactive_textbook.csv,462,96,360,6
9,CollegeAlgCoreq-WEB.csv,477,477,0,0


In [33]:
indices["Kuttler-LinearAlgebra-AFirstCourse-2017A.csv"]

,Unnamed: 0,concept,pages,level,wiki_concept
0,0,∩,537,1,Intersection (set theory)
1,1,∪,537,1,Union (set theory)
2,2,\,537,1,Backslash
3,3,row-echelon form,16,1,Row echelon form
4,4,reduced row-echelon form,16,1,Row echelon form
...,...,...,...,...,...
297,297,work,265,1,Work
298,298,zero matrix,54,1,Zero matrix
299,299,zero subspace,201,1,Linear subspace
300,300,zero transformation,"271, 500",1,Direct-quadrature-zero transformation


In [4]:
import sys
sys.path.append('../src/')
import wikifier
import numpy as np

In [ ]:
batch_size = 5000
index_annotations = {}
for book, index in indices.items():
    print(book)
    indexstr = ".\n".join(index.concept.dropna())
    print(len(indexstr))
    index_annotations[book] = []
    batches = range(int(np.ceil(len(indexstr)/batch_size)))
    for i in batches:
        start = i*batch_size
        end = min((i+1)*batch_size, len(indexstr))
        input = indexstr[start:end]
        print(len(input))
        index_annotations[book].append(wikifier.get_annotations(input, start))

In [5]:
import wikipedia
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')

In [103]:
wiki_concepts = {}
for book, index in list(indices.items()):
    print(book)
    results = [wikipedia.search(concept) for concept in index.concept]
    index['search_results'] = results

Kuttler-LinearAlgebra-AFirstCourse-2017A.csv
Beezer_First_Course.csv
textbook_Hoffman_Kunze.csv
Nicholson-OpenLAWA-2019A.csv
Linear algebra done right — Axler.csv
Math1410_print.csv
Hefferon_LinAlgebra.csv
linear-Cherey, Denton.csv
interactive_textbook.csv
CollegeAlgCoreq-WEB.csv


In [117]:
links = {}

In [131]:
def unique_sorted(pages):
    if type(pages) == "int": return pages
    pages = list(set(pages.split(", ")))
    pages.sort()
    return ", ".join(pages)

def get_link_ratio(candidate, concept_set):
    if not candidate in links:
        candidate_links = ""
        try:
            page = wiki_wiki.page(candidate)
            candidate_links = page.links
        except wikipedia.exceptions.DisambiguationError:
            print("Could not disambiguate: " + candidate)
        except wikipedia.exceptions.PageError:
            print("Could not find page: " + candidate)
        links[candidate] = candidate_links

    num_links = len(links[candidate])
    num_in_set = len(concept_set.intersection(links[candidate]))
    return num_in_set / num_links if num_links > 0 else 0

def get_best_result(results, concept_set):
    if len(results) == 0: return None
    elif len(results) == 1: return results[0]
    link_ratios = np.array([get_link_ratio(candidate, concept_set) for candidate in results])
    filtered_results = list(filter(lambda c: c[1] > 0.1, zip(results, link_ratios)))
    print(filtered_results[0][0] if len(filtered_results) else None)
    return filtered_results[0][0] if len(filtered_results) else None

for book, index in list(indices.items()):
    print(book)
    concept_set = set(sum(index.search_results, []))
    print(book + " with " + str(len(concept_set)) + " potential concepts")
    wiki_concepts = [get_best_result(results, concept_set) for results in index.search_results]
    index['wiki_concept'] = wiki_concepts
    index.to_csv("../dat/index/"+ book)
    index.pages = index.pages.astype(str)
    wiki_index = indices[book].groupby("wiki_concept").agg({'pages': ', '.join})
    wiki_index.pages = wiki_index.pages.apply(unique_sorted)
    wiki_index.to_csv("../dat/index_by_wiki/"+ book)

textbook_Hoffman_Kunze.csv
textbook_Hoffman_Kunze.csv with 1883 potential concepts
Adjugate matrix
Hermitian adjoint
Unitary representation
Algebra
Formal power series
Self-adjoint
Algebraically closed field
Multilinear map
Annihilator (ring theory)
Ideal (ring theory)
Annihilator (ring theory)
Approximation
Associative property
Matrix multiplication
Vector (mathematics and physics)
Augmented matrix
Axiom of countable choice
Basis
Change of basis
Dual basis
Free module
Basis (linear algebra)
Orthonormal basis
Orthonormal basis
Bessel's inequality
Bilinear form
Bilinear form
Orthogonal group
Bilinear form
Degenerate bilinear form
Definite quadratic form
Bilinear form
Symmetric bilinear form
Bilinear form
Symmetric bilinear form
Cauchy–Schwarz inequality
Cayley–Hamilton theorem
Cayley transform
Characteristic (algebra)
Characteristic polynomial
Characteristic polynomial
Eigenvalues and eigenvectors
Characteristic vector
Adjugate matrix
Polynomial
Cofactor
Equivalence
Operation
Rank
Commu

In [132]:
len(links.keys())

11050

In [10]:
path = "../dat/index_by_wiki/"
books = []
wiki_concepts = {}

for book in os.listdir(path):
    index = pd.read_csv(path + book)
    for concept in index.wiki_concept:
        if concept not in wiki_concepts:
            wiki_concepts[concept] = 0
        wiki_concepts[concept] += 1

wiki_concepts = pd.DataFrame({"concept": wiki_concepts.keys(), "occurrences": wiki_concepts.values()})
print(sum(wiki_concepts["occurrences"] >= 5))


107


In [12]:
wiki_concepts[wiki_concepts["occurrences"] >= 5]

,concept,occurrences
1,Absolute value,7
2,Adjugate matrix,5
3,Affine transformation,6
8,Basis,9
9,Basis (linear algebra),5
...,...,...
353,Subset,5
360,Transformation,5
374,Additive inverse,5
401,Block matrix,5
